In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [ ]:
#데이터 셋
X, Y = make_moons(n_samples=500, noise=0.3)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
#보팅
Voting_clf = VotingClassifier(estimators =[('lr', LogisticRegression()), ('rf', RandomForestClassifier()), ('svc', SVC())])

Voting_clf.fit(X_train, Y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [ ]:
#보팅 안에 있는 각 모델의 성능 확인
for n,cls in Voting_clf.named_estimators_.items():
  print(f"{n} = {cls.score(X_test, Y_test)}")

lr = 0.85
rf = 0.92
svc = 0.93


In [ ]:
#Classifier 안에서 의견을 모았을 경우 성능
Voting_clf.score(X_test, Y_test)

0.93

In [ ]:
#동일한 테스트 데이터에 대해 각 분류기와 보팅 분류기가 어떤 의견을 내는지 확인
Voting_clf.predict(X_test[:15])
#1번 데이터셋에 대해 어떻게 생각해 ?

array([1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0])

In [ ]:
[clf.predict(X_test[:1]) for clf in Voting_clf.estimators_]
#보팅 clf 안에 있는 각각의 분류기에게 질문. 1번 데이터셋에 대해 어떻게 생각해?
#1번에 속할 것 같다. #하드보팅 : 다수결

[array([1]), array([1]), array([1])]

In [ ]:
# 소프트 보팅으로 변환
Voting_clf.voting = "soft"

#옵션 변환을 위해 named_esimators로 가져옴 -> probability
Voting_clf.named_estimators["svc"].probability = True

#훈련
Voting_clf.fit(X_train, Y_train)
Voting_clf.score(X_test, Y_test)

0.91

In [ ]:
#배깅 (중복O)

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500, max_samples=100, n_jobs=-1)
#maxsamples는 각 결정 트리가 학습할 샘플의 수, njobs는 cpu를 모두 사용해 병렬 처리. -1은 모든 것을 쓰겠다는 뜻

bag_clf.fit(X_train, Y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=100,
                  n_estimators=500, n_jobs=-1)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = bag_clf.predict(X_test)
accuracy_score(Y_test, y_pred)

0.93

In [ ]:
#oob를 통한 검증 코드
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500, oob_score=True, n_jobs=-1)
bag_clf.fit(X_train, Y_train)
bag_clf.oob_score_

0.8675

In [ ]:
#AdaBoost 활용

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=30, learning_rate=0.5) #전에 오답노트한 것을 새로운 분류기에 얼마나 반영할지

ada_clf.fit(X_train, Y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=30)

In [ ]:
y_pred = ada_clf.predict(X_test)
accuracy_score(Y_test, y_pred)

0.91

In [ ]:
#그래디언트 부스트

In [ ]:
import numpy as np

np.random.seed(42)
X = np.random.rand(100, 1) -0.5
y = 3*X[:, 0] ** 2 + 0.05 * np.random.randn(100)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=0.7)

gbr.fit(X, y)

GradientBoostingRegressor(learning_rate=0.7, max_depth=2, n_estimators=3)

In [ ]:
#그래디언트 부스트가 회귀 선을 추정한 것을 시각화하기 위한 코드

def plot_predictions(reg, X, Y, axes, style, label=None, data_style='b', data_label=None):

  x1 = np.linespace(axes[0], axes[1], 500) #a부터 b까지를 500구간으로 나누기
  y_pred = sum(regr.predict(x1.reshape(-1,1)) for regr in reg) #예측한 것을 더해서 y에 넣기. x와 짝짓기 위해

  plt.plot(X[:,0], y, data_style, label=data_label) #x,y 우리가 넣어준 것을 점찍는데 굵게 표시, 라벨 표시까지 해서 점 찍는 그래프를 만들기
  plt.plot(x1, y_pred, style, linewidth=2, label=label)

  if label or data_label:
    plt.legend(loc="upper center")
    plt.axis(axes)

In [ ]:
# 스태킹

In [ ]:
from sklearn.ensemble import StackingClassifier

stacking_clf = StackingClassifier(estimators=[('lr', LogisticRegression()), ('rf', RandomForestClassifier()), ('svc', SVC(probability=True))], final_estimator = RandomForestClassifier(), cv=5)

stacking_clf.fit(X_train, Y_train)

StackingClassifier(cv=5,
                   estimators=[('lr', LogisticRegression()),
                               ('rf', RandomForestClassifier()),
                               ('svc', SVC(probability=True))],
                   final_estimator=RandomForestClassifier())

In [ ]:
stacking_clf.score(X_test, Y_test)

0.91